# Analysis of Test Metrics

This notebook analyzes the relationship between inventory sums and action sums from test trajectories.

In [1]:
import yaml
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pandas as pd
from glob import glob
import numpy as np
# increase the number of columns displayed
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 1000)
# from sklearn import linear_model
# from plotter import *
# from ipywidgets import *
# import statsmodels.api as sm
# from plotter import *
# from matplotlib.ticker import StrMethodFormatter
# import matplotlib.pyplot as plt
# import matplotlib.patches as mpatches
# from matplotlib.lines import Line2D
# import scipy.stats as stats



In [2]:
def load_metrics_files(date=None, filename=None, base_path='../metrics/test_trajectories'):
    """
    Load metrics files from the specified path
    
    Args:
        date (str): Date in format 'YYYY_MM_DD'. If None, uses latest date
        filename (str, optional): Specific filename to load. If None, loads latest file from date
        base_path (str): Base path for metrics files
    """
    if date is None:
        # Get all date folders and select the latest
        date_folders = glob(os.path.join(base_path, '*_*_*'))
        if not date_folders:
            raise ValueError("No date folders found")
        date = max(os.path.basename(folder) for folder in date_folders)
    
    # Construct path with date
    date_path = os.path.join(base_path, date)
    
    if filename:
        # Load specific file
        metrics_files = [os.path.join(date_path, filename)]
    else:
        # Find all CSV files for the date and get the latest one
        metrics_files = glob(os.path.join(date_path, '**', '*_test_metrics.csv'), recursive=True)
        if not metrics_files:
            raise ValueError(f"No metrics files found for date {date}")
        metrics_files = [max(metrics_files, key=os.path.getctime)]
    
    # Read and concatenate all CSV files
    dfs = []
    for file_path in metrics_files:
        if os.path.exists(file_path):
            print(f"Loading metrics from: {file_path}")
            df = pd.read_csv(file_path)
            dfs.append(df)
        else:
            print(f"File not found: {file_path}")
    
    return pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()



# Load specific file from specific date
# df = load_metrics_files(date='2024_02_14', filename='model123_test_metrics.csv')

# Load latest file from latest date
# df = load_metrics_files()

In [ ]:
# Example usage:
# Load latest file from specific date
# df = load_metrics_files(date='2025_02_14')
df = load_metrics_files(date='2025_03_18', filename='one_store_fixed_costs/hybrid_policy/1742319349_876_test_data.csv')
# df = load_metrics_files(date='2025_03_11', filename='one_store_fixed_costs/hybrid_policy/1741734767_249_test_data.csv')
# df = load_metrics_files(date='2025_03_10', filename='one_store_fixed_costs/hybrid_policy/1741641791_81_test_data.csv')
# df = load_metrics_files(date='2025_03_07', filename='one_store_fixed_costs/hybrid_policy/1741405593_624_test_data.csv')
# df = load_metrics_files(date='2025_03_06', filename='one_store_fixed_costs/hybrid_policy/1741242921_461_test_data.csv')
# df = load_metrics_files(date='2025_03_05', filename='one_store_fixed_costs/hybrid_policy/1741236758_234_test_data.csv')
# df = load_metrics_files(date='2025_03_05', filename='one_store_fixed_costs/hybrid_policy/1741205699_438_test_data.csv')
# df = load_metrics_files(date='2025_03_05', filename='one_store_fixed_costs/hybrid_policy/1741212087_313_test_data.csv')
# df = load_metrics_files(date='2025_03_05', filename='one_store_fixed_costs/hybrid_policy/1741207298_293_test_data.csv')
# df = load_metrics_files(date='2025_03_04', filename='one_store_fixed_costs/hybrid_policy/1741103211_609_test_data.csv')
# df = load_metrics_files(date='2025_03_03', filename='one_store_fixed_costs/hybrid_policy/1741027502_875_test_data.csv')
# df = load_metrics_files(date='2025_03_03', filename='one_store_fixed_costs/hybrid_policy/1741027503_554_test_data.csv')
# df = load_metrics_files(date='2025_03_03', filename='one_store_fixed_costs/hybrid_policy/1741018361_69_test_data.csv')
# df = load_metrics_files(date='2025_03_03', filename='one_store_fixed_costs/hybrid_policy/1741018361_69_test_data_raw.pt')
# df = load_metrics_files(date='2025_03_02', filename='one_store_fixed_costs/hybrid_policy/1740967706_828_test_data.pt')
# df = load_metrics_files(date='2025_03_03', filename='one_store_fixed_costs/hybrid_policy/1741011238_374_test_metrics.csv')
# df = load_metrics_files(date='2025_02_19', filename='one_store_fixed_costs/hybrid_policy/1739983192_910_test_metrics.csv')
# df = load_metrics_files(date='2025_02_19', filename='one_store_fixed_costs/hybrid_policy/1739983192_74_test_metrics.csv')
# df = load_metrics_files(date='2025_02_19', filename='one_store_fixed_costs/hybrid_policy/1739989165_590_test_metrics.csv')
# sort by batch_idx then by time_step
df = df.sort_values(by=['batch_idx', 'time_step'])
# df = load_metrics_files(date='2025_02_17', filename='one_store_fixed_costs/hybrid_policy/1739815253_test_metrics.csv')
# df = load_metrics_files(date='2025_02_17', filename='one_store_fixed_costs/hybrid_policy/1739810390_test_metrics.csv')
# df = load_metrics_files(date='2025_02_17', filename='one_store_fixed_costs/hybrid_policy/1739776323_test_metrics.csv')
# df = load_metrics_files(date='2025_02_17', filename='one_store_fixed_costs/hybrid_policy/1739779365_test_metrics.csv')
# df = load_metrics_files(date='2025_02_17', filename='one_store_fixed_costs/hybrid_policy/1739776205_test_metrics.csv')
# df = load_metrics_files(date='2025_02_16', filename='one_store_fixed_costs/hybrid_policy/1739768092_test_metrics.csv')
# /user/ma4177/Exp_neural/metrics/test_trajectories/2025_02_17/one_store_fixed_costs/hybrid_policy/1739779638_test_metrics.csv
df.head()

# len(df)

In [4]:
# create new column by applying softmax to raw_discrete_logits
df['softmax_discrete_logits'] = df['discrete'].apply(lambda x: np.exp(np.array(eval(x), dtype=np.float64)) / np.sum(np.exp(np.array(eval(x), dtype=np.float64))))
df['total_action_softmaxed'] = df['total_action'] * df['softmax_discrete_logits'].apply(lambda x: x[1] if isinstance(x, np.ndarray) and len(x) > 1 else 0)


In [ ]:
# print a random sub-sample of 100 rows
df.sample(10)

In [6]:
# compute -5 + 67*sigmoid(continuous)
df['continuous_scaled'] = -5 + 67*df['continuous'].apply(lambda x: 1 / (1 + np.exp(-float(eval(x)[0]))))


In [ ]:
# make a histogram of the continuous_scaled
plt.hist(df['continuous_scaled'], bins=100)
plt.show()


In [ ]:
import matplotlib.pyplot as plt

print(len(df))
# Extract the last index of softmax_discrete_logits
last_index_values = df['softmax_discrete_logits'].apply(lambda x: x[-1] if isinstance(x, np.ndarray) and len(x) > 1 else 0)

# Calculate the percentage of values that are almost discrete
almost_discrete_count = ((last_index_values < 0.1) | (last_index_values > 0.9)).sum()
percentage_almost_discrete = (almost_discrete_count / len(last_index_values)) * 100
print(f'Percentage of almost discrete values: {percentage_almost_discrete:.2f}%')

# Calculate percertage smaller than 0.5
percentage_smaller_than_0_5 = (last_index_values < 0.5).sum() / len(last_index_values) * 100
# Create a histogram of the last index values
plt.figure(figsize=(10, 6))
plt.hist(last_index_values, bins=30, alpha=0.7, color='blue')
plt.title('Histogram of Last Index of Softmax Discrete Logits')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.grid(axis='y', alpha=0.75)
plt.show()


In [ ]:
discrete_action_index = 1
value = 'continuous_values'

# Filter the DataFrame for discrete_action_index = 1
filtered_df = df[df['discrete_action_index'] == discrete_action_index]

# Extract the last index of continuous_values after converting the string to a tuple
last_index_continuous_values = filtered_df[value].apply(lambda x: eval(x)[-1] if isinstance(eval(x), tuple) and len(eval(x)) > 1 else 0)
print(last_index_continuous_values)

# Create a histogram of the last index values
plt.figure(figsize=(10, 6))
plt.hist(last_index_continuous_values, bins=30, alpha=0.7, color='green')
plt.title('Histogram of Last Index of Continuous Values (discrete_action_index=1)')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.grid(axis='y', alpha=0.75)
plt.show()


In [ ]:
df = load_metrics_files(date='2025_02_26', filename='one_store_fixed_costs/hybrid_policy/1740571952_838_test_metrics.csv')
df2 = load_metrics_files(date='2025_04_15', filename='one_store_fixed_costs/hybrid_policy/1744717431_555_test_data.csv')
# df = load_metrics_files(date='2025_04_14', filename='one_store_fixed_costs/factored_policy/1744653950_997_test_data.csv')
# df = load_metrics_files(date='2025_03_18', filename='one_store_fixed_costs/hybrid_policy/1742338049_683_test_data.csv')
# df = load_metrics_files(date='2025_03_11', filename='one_store_fixed_costs/hybrid_policy/1741710362_302_test_data.csv')
# df = load_metrics_files(date='2025_03_11', filename='one_store_fixed_costs/hybrid_policy/1741743750_608_test_data.csv')
# df = load_metrics_files(date='2025_03_02', filename='one_store_fixed_costs/hybrid_policy/1740938929_431_test_metrics.csv')
# df = load_metrics_files(date='2025_02_28', filename='one_store_fixed_costs/hybrid_policy/1740787963_719_test_metrics.csv')
# df = load_metrics_files(date='2025_02_26', filename='one_store_fixed_costs/hybrid_policy/1740593892_53_test_metrics.csv')
# df = load_metrics_files(date='2025_02_26', filename='one_store_fixed_costs/hybrid_policy/1740577006_900_test_metrics.csv')
# df = load_metrics_files(date='2025_02_25', filename='one_store_fixed_costs/hybrid_policy/1740535848_304_test_metrics.csv')
# df = load_metrics_files(date='2025_02_19', filename='one_store_fixed_costs/hybrid_policy/1739993263_778_test_metrics.csv')
# df = load_metrics_files(date='2025_02_19', filename='one_store_fixed_costs/hybrid_policy/1739998287_368_test_metrics.csv')
# df = load_metrics_files(date='2025_02_19', filename='one_store_fixed_costs/hybrid_policy/1739997478_625_test_metrics.csv')
# df = load_metrics_files(date='2025_02_19', filename='one_store_fixed_costs/hybrid_policy/1739992488_876_test_metrics.csv')
# df = load_metrics_files(date='2025_02_19', filename='one_store_fixed_costs/hybrid_policy/1739989165_590_test_metrics.csv')
# sort by batch_idx then by time_step
df = df.sort_values(by=['batch_idx', 'time_step'])

# in df, if there is a column called total_action_sum, replace it with total_action
df.rename(columns={'total_action_sum': 'total_action'}, inplace=True)
# df2.rename(columns={'total_action_sum': 'total_action'}, inplace=True)
print(df.head())
print(df2.head())


In [ ]:
print(100*(37.13 - 37.06)/37.06)

In [ ]:
df = load_metrics_files(date='2025_02_26', filename='one_store_fixed_costs/hybrid_policy/1740577006_900_test_metrics.csv')
# df = load_metrics_files(date='2025_04_14', filename='one_store_fixed_costs/hybrid_policy/1744665202_443_test_data.csv')
# df = load_metrics_files(date='2025_04_15', filename='one_store_fixed_costs/hybrid_policy/1744752149_38_test_data.csv')
# df = load_metrics_files(date='2025_02_25', filename='one_store_fixed_costs/hybrid_policy/1740539256_871_test_metrics.csv')
df.rename(columns={'total_action_sum': 'total_action'}, inplace=True)
# df = df.sort_values(by=['batch_idx', 'time_step'])

# %matplotlib notebook
%matplotlib ipympl
plt.clf()
# Set up the plot style
plt.figure(figsize=(10, 6))

# Choose a random sub-sample of n points for the plot
n = 1000  # Specify the number of points to sample
sampled_df = df.sample(n=n, random_state=1)  # Randomly sample n points

# Create scatter plot with smaller dots
sns.scatterplot(data=sampled_df, x='inventory_sum', y='total_action', alpha=0.6, palette='bright', s=30)  # Adjusted size of dots
# sns.scatterplot(data=sampled_df, x='inventory_sum', y='total_action', alpha=0.6, palette='bright', s=30)  # Adjusted size of dots
plt.title('Relationship between Inventory Sums and Action Sums (Sampled)')
plt.xlabel('Total Inventory')
plt.ylabel('Total Actions')

# Plot (s, S) policy line
s, S = 26, 62
inventory_range = np.linspace(df['inventory_sum'].min(), df['inventory_sum'].max(), 100)
order_amounts = np.maximum(S - inventory_range, 0) * (inventory_range <= s)

plt.plot(inventory_range, order_amounts, color='red', label='(s, S) Policy: Order Amount')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# %matplotlib notebook
%matplotlib ipympl
plt.clf()
# Set up the plot style
plt.figure(figsize=(10, 6))

# Create a hexbin plot to show the density of points
plt.hexbin(df['inventory_sum'], df['total_action'], gridsize=30, cmap='Greens', mincnt=1)
plt.colorbar(label='Counts in bin')

# Plot (s, S) policy line
s, S = 26, 62
inventory_range = np.linspace(df['inventory_sum'].min(), df['inventory_sum'].max(), 100)
order_amounts = np.maximum(S - inventory_range, 0) * (inventory_range <= s)

plt.plot(inventory_range, order_amounts, color='red', label='(s, S) Policy: Order Amount')
plt.title('Relationship between Inventory Sums and Action Sums (Hexbin)')
plt.xlabel('Total Inventory')
plt.ylabel('Total Actions')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Create order boolean based on the condition
df['order_boolean'] = (df['total_action'] >= 0.1).astype(int)

# Bucketize inventory_sum into bins of size 1
df['inventory_bucket'] = (df['inventory_sum'] // 1).astype(int)

# Calculate the proportion of pairs for each (inventory_bucket, order_boolean) pair
heatmap_data = df.groupby(['inventory_bucket', 'order_boolean']).size().unstack(fill_value=0)
heatmap_data = heatmap_data.div(heatmap_data.sum().sum(), axis=0)  # Normalize to get proportions across all cells

# Plot the heatmap
plt.figure(figsize=(12, 6))
sns.heatmap(heatmap_data, cmap='YlGnBu', annot=False, cbar_kws={'label': 'Proportion'})  # Remove annotations
plt.title('Heatmap of Inventory Sum vs Order Boolean')
plt.xlabel('Order Boolean (0: Order < 0.1, 1: Order >= 0.1)')
plt.ylabel('Inventory Sum (Bucketized)')
plt.show()


In [ ]:
# Calculate the average order for each (inventory_bucket, order_boolean) pair
heatmap_data = df.groupby(['inventory_bucket', 'order_boolean'])['total_action'].mean().unstack(fill_value=0)

# Plot the heatmap
plt.figure(figsize=(12, 6))
sns.heatmap(heatmap_data, cmap='YlGnBu', annot=True, cbar_kws={'label': 'Average Order'})  # Add annotations for average order
plt.title('Heatmap of Inventory Sum vs Average Order')
plt.xlabel('Order Boolean (0: Order < 0.1, 1: Order >= 0.1)')
plt.ylabel('Inventory Sum (Bucketized)')
plt.show()

In [ ]:
# make a histogram of the inventory_sum
plt.cla()
plt.hist(df['inventory_on_hand'], bins=100)
# plt.hist(df['inventory_sum'], bins=100)
plt.show()
# # make a histogram of the action_sum
# plt.hist(df['total_action_sum'], bins=100)
# plt.show()


In [ ]:
# Calculate correlation
correlation = df.groupby('model_name').apply(lambda x: x['inventory_sum'].corr(x['action_sum']))
print("\nCorrelation between inventory and actions by model:")
print(correlation)

In [ ]:
# Basic statistics
print("\nSummary statistics by model:")
stats = df.groupby('model_name').agg({
    'inventory_sum': ['mean', 'std', 'min', 'max'],
    'action_sum': ['mean', 'std', 'min', 'max']
})
stats